In [ ]:
dbutils.fs.unmount("/mnt/s3data")

In [ ]:
import urllib
ACCESS_KEY = ""
SECRET_KEY = ""
ENCODED_SECRET_KEY = urllib.parse.quote(SECRET_KEY, "")
AWS_BUCKET_NAME = ""
MOUNT_NAME = "s3data"
dbutils.fs.mount("s3n://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)

In [ ]:
display(dbutils.fs.ls("/mnt/s3data"))

In [ ]:
# File location and type
file_location = "dbfs:/mnt/s3data/listing.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_listing = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_listing)

In [ ]:
# File location and type
file_location = "dbfs:/mnt/s3data/listing.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_listing = spark.read.format(file_type) \
  .option("multiline","true") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_listing)


In [ ]:
# File location and type
file_location = "dbfs:/mnt/s3data/listing.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_listing = spark.read.format(file_type) \
  .option("multiline","true") \
  .option("quote", "\"") \
  .option("escape", "\"") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_listing)

In [ ]:
%sql

drop table listing

In [ ]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

listing_table_name = "listing"

df_listing.write.format("parquet").saveAsTable(listing_table_name)

In [ ]:
df_listing.printSchema()

In [ ]:
from pyspark.sql.functions import *

df_listing=df_listing.withColumn("scrape_id",col("scrape_id").cast(DoubleType()))

In [ ]:
df_listing.printSchema()

In [ ]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

listing_table_name = "listing"

df_listing.write.format("parquet").saveAsTable(listing_table_name)

In [ ]:
%sql
select * from listing

In [ ]:
from pyspark.sql.types import *

# File location and type
file_location = "dbfs:/mnt/s3data/neighbourhoods.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

neighbourhoodSchema = StructType([
    StructField("neighbourhood_group", StringType(), True),        
    StructField("neighbourhood", StringType(), True)
])

# The applied options are for CSV files. For other file types, these will be ignored.
df_neighbourhoods = spark.read.format(file_type) \
  .option("wholeFile", "true") \
  .option("multiline","true") \
  .option("schema", neighbourhoodSchema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_neighbourhoods)


In [ ]:
listing_table_name = "neighbourhood"

df_neighbourhoods.write.mode("overwrite").format("parquet").saveAsTable(listing_table_name)

In [ ]:
# File location and type
file_location = "dbfs:/mnt/s3data/reviews.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_reviews = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_reviews)

In [ ]:
# File location and type
file_location = "dbfs:/mnt/s3data/reviews.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_reviews = spark.read.format(file_type) \
  .option("wholeFile", "true") \
  .option("multiline","true") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_reviews)

In [ ]:
display(spark.read.text("dbfs:/mnt/s3data/reviews.csv"))

In [ ]:
temp_df=spark.read.text("dbfs:/mnt/s3data/reviews.csv")

In [ ]:
header=temp_df.head(1)

In [ ]:
fixed_df=temp_df.filter(temp_df.value.contains("/20"))

In [ ]:
fixed_df.count()

In [ ]:
display(fixed_df)

In [ ]:
final_df=spark.createDataFrame(header).union(fixed_df)

In [ ]:
final_df.show()

In [ ]:
final_df.repartition(1).write.text("dbfs:/mnt/s3data/reviews_fixed")

In [ ]:
display(dbutils.fs.ls("/mnt/s3data"))

In [ ]:
# File location and type
file_location = "dbfs:/mnt/s3data/reviews_fixed/"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_reviews = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_reviews)

In [ ]:
listing_table_name = "reviews"

df_reviews.write.format("parquet").saveAsTable(listing_table_name)

In [ ]:
%sql

select * from listing

In [ ]:
display(df_listing.groupBy('property_type').count())

In [ ]:
from pyspark.sql.functions import *

df_listing.groupBy('property_type').count().orderBy(col('count').desc()).show()

In [ ]:
df_listing.groupBy("property_type","price").agg(({"price": "average"})).show()

In [ ]:
%sql
select distinct price from listing

In [ ]:
from pyspark.sql.types import FloatType

def trim_char(string):
    return string.strip('$')

spark.udf.register("trim_func", trim_char)

trim_func_udf = udf(trim_char)


In [ ]:
df_listing.select("property_type","price",trim_func_udf("price").cast(FloatType()).alias("price_f")).groupBy("property_type").agg(({"price_f":"average"})).show()

In [ ]:
df_listing.select("property_type","price",trim_func_udf("price").cast(FloatType()).alias("price_f")).groupBy("property_type").agg(({"price_f":"average"})).show()